In [1]:
import requests
import sys
from helpers_humana import JSONOpen
from tqdm.notebook import tqdm
import gzip
import csv

In [2]:
import ijson

def gen_in_network_links(index_loc):
    """
    Gets in-network files from index.json files
    :param index.json URL:
    """
    with JSONOpen(index_loc) as f:
        count = 0
        parser = ijson.parse(f, use_float=True)
        for prefix, event, value in parser:
            if (
                prefix.endswith('location')
                and event == 'string'
                and 'in-network' in value
            ):
                yield value
                count += 1

In [3]:
unique_lines = set()
unique_files = set()
with open("files.txt", "r") as f:
    with open ("F:\\_bounty\\humana_payers.csv", "a", newline="") as output_csv:
        writer = csv.DictWriter(output_csv, fieldnames=["url"])
        # writer.writeheader()
        for filename in tqdm(f, total=9437):
            # Index files, needs to a url
            if filename not in unique_files:
                unique_files.add(filename)
                filename = "https://developers.humana.com/syntheticdata/Resource/DownloadTOCFile?&fileName=" + filename.strip("\n")
                # filename = JSONOpen(filename)
                try:
                    for url in gen_in_network_links(filename):
                        if url not in unique_lines:
                            unique_lines.add(url)
                            writer.writerow({
                                "url": url.replace("https://developers.humana.com/Resource/DownloadPCTFile", "") # reduced output size by ~3gb
                            })
                            
                except KeyboardInterrupt:
                    break
                except:
                    pass

  0%|          | 0/9437 [00:00<?, ?it/s]